In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
from model.net import FashionNet
from load_dataset import custom_dset, collate_fn, denorm
from torch.utils.data import DataLoader
import cv2
import argparse
from utils import AverageMeter, AverageAcc
from tqdm import tqdm
from focal_loss import *


In [2]:
from IPython.display import display, HTML
import pandas as pd

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [3]:
def test(net, test_loader, accuracy):


    losses = AverageMeter()
    criterion = FocalLoss(gamma=2, alpha=0.25)
    bar = tqdm(total=len(test_loader))

    with torch.no_grad():
        for it, (imgs, labels, img_ids) in enumerate(test_loader):
            bar.update()
            imgs = imgs.cuda()
            labels = labels.cuda()

            # run input through the network
            preds = net(imgs)
            loss = criterion(preds, labels)


            # measure accuracy and record loss
            accuracy.update(preds.data, labels.data, topk=(1, 5))
            losses.update(loss.item(), imgs.size(0))

            if it % 100 == 0:
                # save to disk first batch of results for every epoch
                for i,im in enumerate(imgs):
                    im = im.data.cpu().numpy()
                    im = denorm(im.transpose(1,2,0))[...,::-1]

                    _, inds = torch.max(preds, 1)
                    pred = label_map[inds[i].item()]
                    gt_label = label_map[labels[i].item()]

                    fname =  str(img_ids[i]) + '__' 'Gt:' + gt_label + 'Pred:' + pred + '.jpg'
                    cv2.imwrite(os.path.join(out_path, fname),  im)

    bar.close()


In [4]:
# check checkpoints folder for model_name
model_name = 'finetuned_4.pth'


# get the current working directory
root = os.getcwd()
out_path = './data/show_test'
if not os.path.isdir(out_path):
    os.mkdir(out_path)


# load model
model_path = os.path.join(root, 'checkpoints', model_name)
model = torch.load(model_path)
label_map = model['label_map']
test_finetune = custom_dset('test_finetune', label_map)
net = FashionNet(len(label_map))
net.load_state_dict(model['net'])

net = net.cuda()
net.eval()


label_map = { v:k for k,v in label_map.items()}


print('Pretrain Test Dataset of size: {}'.format(len(test_finetune)))
test_loader = DataLoader(test_finetune, batch_size=28, shuffle=False, collate_fn=collate_fn, num_workers=4)

accuracy = AverageAcc(test_finetune.label_map)
test(net, test_loader, accuracy)

# get accuracies per class and the averages 
topk_dict, top1, top5 = accuracy()


  0%|          | 0/112 [00:00<?, ?it/s]

Pretrain Test Dataset of size: 3135


100%|██████████| 112/112 [00:08<00:00, 11.53it/s]


In [5]:
# collect accuracies into lists for printing purpose 
lc = [k for k in test_finetune.label_map]
t1 = [v[1] for k,v in topk_dict.items()]
t2 = [v[5] for k,v in topk_dict.items()]
avg = [round(top1, 2), round(top5, 2)]

In [6]:
trainset =  custom_dset('train_finetune')
lc_train = [trainset.freq[c] for c in lc]

# display train set statitstics 
df_dset = pd.DataFrame(np.vstack((lc, lc_train)).transpose(), index=['Trainset']*66, columns=['Category', 'Data examples'])
display(df_dset)

# display test set statitstics 
df_dset = pd.DataFrame(np.vstack( (lc, [v[1] for _,v in accuracy.count.items()])).transpose(), index=['Testset']*66, columns=['Category', 'Data examples'])
display(df_dset)

# concatenate accuracy lists in numpy array of shape (num_classes, num_columns)
df_class = pd.DataFrame(np.vstack( (lc, t1, t2)).transpose(), columns=['Category', 'Top1', 'Top5'], index=['TestAcc']*66)
df_avg = pd.DataFrame(np.array(avg).reshape(1,2), columns=['Average Top1', 'Average Top5'])

# display the accuracies 
display(df_class)



,Category,Data examples
Trainset,Accessory Gift Set,12
Trainset,Bangle,54
Trainset,Basketballs,2
Trainset,Bra,131
Trainset,Bracelet,33
Trainset,Camisoles,3
Trainset,Capris,102
Trainset,Caps,131
Trainset,Churidar,25
Trainset,Clutches,188


,Category,Data examples
Testset,Accessory Gift Set,91
Testset,Bangle,20
Testset,Basketballs,11
Testset,Bra,259
Testset,Bracelet,19
Testset,Camisoles,30
Testset,Capris,42
Testset,Caps,110
Testset,Churidar,4
Testset,Clutches,50


,Category,Top1,Top5
TestAcc,Accessory Gift Set,0.98,1.0
TestAcc,Bangle,0.05,0.8
TestAcc,Basketballs,0.0,0.0
TestAcc,Bra,0.95,0.97
TestAcc,Bracelet,0.63,1.0
TestAcc,Camisoles,0.0,0.0
TestAcc,Capris,0.33,0.95
TestAcc,Caps,0.79,0.88
TestAcc,Churidar,0.5,0.75
TestAcc,Clutches,0.96,0.98


In [7]:


# concatenate accuracy lists in numpy array of shape (num_classes, num_columns)
df_class = pd.DataFrame(np.vstack( (lc, t1, t2)).transpose(), columns=['Category', 'Top1', 'Top5'], index=['TestAcc']*66)
df_avg = pd.DataFrame(np.array(avg).reshape(1,2), columns=['Average Top1', 'Average Top5'], index=['training'])

# display the accuracies 
display(df_class)
display(df_avg)

,Category,Top1,Top5
TestAcc,Accessory Gift Set,0.98,1.0
TestAcc,Bangle,0.05,0.8
TestAcc,Basketballs,0.0,0.0
TestAcc,Bra,0.95,0.97
TestAcc,Bracelet,0.63,1.0
TestAcc,Camisoles,0.0,0.0
TestAcc,Capris,0.33,0.95
TestAcc,Caps,0.79,0.88
TestAcc,Churidar,0.5,0.75
TestAcc,Clutches,0.96,0.98


,Average Top1,Average Top5
training,0.35,0.65


In [ ]:

sorted(test_finetune.label_map, key=str.lower)


In [12]:
df_class


,Category,Top1,Top5
TestAcc,Accessory Gift Set,0.98,1.0
TestAcc,Bangle,0.05,0.8
TestAcc,Basketballs,0.0,0.0
TestAcc,Bra,0.95,0.97
TestAcc,Bracelet,0.63,1.0
TestAcc,Camisoles,0.0,0.0
TestAcc,Capris,0.33,0.95
TestAcc,Caps,0.79,0.88
TestAcc,Churidar,0.5,0.75
TestAcc,Clutches,0.96,0.98


In [17]:
pd.concat([df_dset, df_class], ignore_index=False, sort=False, join='outer')


,Category,Data examples,Top1,Top5
Testset,Accessory Gift Set,91,NaN,NaN
Testset,Bangle,20,NaN,NaN
Testset,Basketballs,11,NaN,NaN
Testset,Bra,259,NaN,NaN
Testset,Bracelet,19,NaN,NaN
Testset,Camisoles,30,NaN,NaN
Testset,Capris,42,NaN,NaN
Testset,Caps,110,NaN,NaN
Testset,Churidar,4,NaN,NaN
Testset,Clutches,50,NaN,NaN
